In [1]:
%matplotlib inline
# General libraries.
import re
import numpy as np
import matplotlib.pyplot as plt
import csv

# SK-learn libraries for learning.
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.grid_search import GridSearchCV

# SK-learn libraries for evaluation.
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer 



In [2]:
### Load data files

with open('train.csv', 'rb') as f:
    train = list(csv.reader(f))
print train[0]
print train[1]

with open('test.csv', 'rb') as f:
    test = list(csv.reader(f))
print test[0]
print test[1]

with open('sampleSubmission.csv', 'rb') as f:
    sampleSubmission = list(csv.reader(f))
print sampleSubmission[0]
print sampleSubmission[1]



['Dates', 'Category', 'Descript', 'DayOfWeek', 'PdDistrict', 'Resolution', 'Address', 'X', 'Y']
['2015-05-13 23:53:00', 'WARRANTS', 'WARRANT ARREST', 'Wednesday', 'NORTHERN', 'ARREST, BOOKED', 'OAK ST / LAGUNA ST', '-122.425891675136', '37.7745985956747']
['Id', 'Dates', 'DayOfWeek', 'PdDistrict', 'Address', 'X', 'Y']
['0', '2015-05-10 23:59:00', 'Sunday', 'BAYVIEW', '2000 Block of THOMAS AV', '-122.39958770418998', '37.7350510103906']
['Id', 'ARSON', 'ASSAULT', 'BAD CHECKS', 'BRIBERY', 'BURGLARY', 'DISORDERLY CONDUCT', 'DRIVING UNDER THE INFLUENCE', 'DRUG/NARCOTIC', 'DRUNKENNESS', 'EMBEZZLEMENT', 'EXTORTION', 'FAMILY OFFENSES', 'FORGERY/COUNTERFEITING', 'FRAUD', 'GAMBLING', 'KIDNAPPING', 'LARCENY/THEFT', 'LIQUOR LAWS', 'LOITERING', 'MISSING PERSON', 'NON-CRIMINAL', 'OTHER OFFENSES', 'PORNOGRAPHY/OBSCENE MAT', 'PROSTITUTION', 'RECOVERED VEHICLE', 'ROBBERY', 'RUNAWAY', 'SECONDARY CODES', 'SEX OFFENSES FORCIBLE', 'SEX OFFENSES NON FORCIBLE', 'STOLEN PROPERTY', 'SUICIDE', 'SUSPICIOUS OCC'

In [9]:
### This function extracts the selected features from the data files
def features(data, date_pos, dayofweek_pos, pddistric_pos, address_pos):
    
    year, month,day, hour, minutes, dayofweek, districts, streets,  x, y, labels = [],[],[],[],[],[],[],[],[],[],[]

    for i in range(1,len(data)):
        temp_data = []
        temp_data = data[i]
        year.append(temp_data[date_pos].split()[0].split("-")[0])
        month.append(temp_data[date_pos].split()[0].split("-")[1])
        day.append(temp_data[date_pos].split()[0].split("-")[2])
        hour.append(temp_data[date_pos].split()[1].split(":")[0])
        minutes.append(temp_data[date_pos].split()[1].split(":")[1])   
        dayofweek.append(temp_data[dayofweek_pos])
        districts.append(temp_data[pddistric_pos])
        streets.append(temp_data[address_pos])

    features = [year, month, day, hour, minutes, dayofweek, districts, streets]
    return features


##labels.append(temp_data[1])

In [10]:
train_features = features(train,0, 3, 4,6)
for i in range (len(train_features)):
    print train_features[i][0]

2015
05
13
23
53
Wednesday
NORTHERN
OAK ST / LAGUNA ST


In [11]:
test_features = features(test,1,2,3,4)
for i in range (len(test_features)):
    print test_features[i][0]

2015
05
10
23
59
Sunday
BAYVIEW
2000 Block of THOMAS AV


In [29]:
## This function vectorizes the features on both the train and the test data
def vector_features_lists(train_features, test_features):
    vectorizer = CountVectorizer(min_df=1)
    train_vector_features, test_vector_features, feature_labels = [], [], []
    for i in range (len(train_features)):
        train_vector_features.append(vectorizer.fit_transform(train_features[i]))
        test_vector_features.append(vectorizer.transform(test_features[i]))
        feature_labels.append(vectorizer.get_feature_names())
    vector_features_lists = [train_vector_features, test_vector_features, feature_labels]
    return vector_features_lists






In [30]:
lists = vector_features_lists(train_features, test_features)




In [31]:
train_lists = lists[0] 
test_lists = lists[1]
feature_labels = lists[2]

In [41]:
### This function converts the vectorized features into an array
def vector_features_array(vector_features_list):
    for i in range (len(vector_features_list)):
        array = vector_features_list[i].toarray()
        if i == 0:
            vector_features_array = array
        else:
            vector_features_array = np.concatenate([vector_features_array, array], axis = -1)
    
    return vector_features_array


In [42]:
X_data = vector_features_array(train_lists)
X_test = vector_features_array(test_lists)

In [43]:
print len(feature_labels)
print feature_labels[0:100]

8
[[u'2003', u'2004', u'2005', u'2006', u'2007', u'2008', u'2009', u'2010', u'2011', u'2012', u'2013', u'2014', u'2015'], [u'01', u'02', u'03', u'04', u'05', u'06', u'07', u'08', u'09', u'10', u'11', u'12'], [u'01', u'02', u'03', u'04', u'05', u'06', u'07', u'08', u'09', u'10', u'11', u'12', u'13', u'14', u'15', u'16', u'17', u'18', u'19', u'20', u'21', u'22', u'23', u'24', u'25', u'26', u'27', u'28', u'29', u'30', u'31'], [u'00', u'01', u'02', u'03', u'04', u'05', u'06', u'07', u'08', u'09', u'10', u'11', u'12', u'13', u'14', u'15', u'16', u'17', u'18', u'19', u'20', u'21', u'22', u'23'], [u'00', u'01', u'02', u'03', u'04', u'05', u'06', u'07', u'08', u'09', u'10', u'11', u'12', u'13', u'14', u'15', u'16', u'17', u'18', u'19', u'20', u'21', u'22', u'23', u'24', u'25', u'26', u'27', u'28', u'29', u'30', u'31', u'32', u'33', u'34', u'35', u'36', u'37', u'38', u'39', u'40', u'41', u'42', u'43', u'44', u'45', u'46', u'47', u'48', u'49', u'50', u'51', u'52', u'53', u'54', u'55', u'56', u'5

In [44]:
print X_data.shape
print X_test.shape


(878049, 2298)
(884262, 2298)


In [45]:
### This function extracts the data labels from the train data
def labels(data):
    
    labels = []

    for i in range(1,len(data)):
        temp_data = []
        temp_data = data[i]
        labels.append(temp_data[1])

    return labels

labels = labels(train)

classifications = sorted(set(labels))
print classifications
print len(classifications)
print len(sampleSubmission[0])


['ARSON', 'ASSAULT', 'BAD CHECKS', 'BRIBERY', 'BURGLARY', 'DISORDERLY CONDUCT', 'DRIVING UNDER THE INFLUENCE', 'DRUG/NARCOTIC', 'DRUNKENNESS', 'EMBEZZLEMENT', 'EXTORTION', 'FAMILY OFFENSES', 'FORGERY/COUNTERFEITING', 'FRAUD', 'GAMBLING', 'KIDNAPPING', 'LARCENY/THEFT', 'LIQUOR LAWS', 'LOITERING', 'MISSING PERSON', 'NON-CRIMINAL', 'OTHER OFFENSES', 'PORNOGRAPHY/OBSCENE MAT', 'PROSTITUTION', 'RECOVERED VEHICLE', 'ROBBERY', 'RUNAWAY', 'SECONDARY CODES', 'SEX OFFENSES FORCIBLE', 'SEX OFFENSES NON FORCIBLE', 'STOLEN PROPERTY', 'SUICIDE', 'SUSPICIOUS OCC', 'TREA', 'TRESPASS', 'VANDALISM', 'VEHICLE THEFT', 'WARRANTS', 'WEAPON LAWS']
39
40


In [46]:
Y_labels = np.zeros((len(labels),))
print Y_labels.shape
for i in range (Y_labels.shape[0]):
    for j in range (len(classifications)):
        if labels[i] == classifications[j]:
                     Y_labels[i] = j
print labels[0]
print Y_labels[0]

(878049,)
WARRANTS
37.0


In [49]:
# Shuffle the data, but make sure that the features and accompanying labels stay in sync.
np.random.seed(0)
shuffle = np.random.permutation(np.arange(X_data.shape[0]))
X_data, Y_labels = X_data[shuffle], Y_labels[shuffle]



In [65]:
# Split into train and dev.
train_data, train_labels = X_data[:700000], Y_labels[:700000]
dev_data, dev_labels = X_data[700000:], Y_labels[700000:]
mini_train_data, mini_train_labels = train_data[:50000], train_labels[:50000]
mini_dev_data, mini_dev_labels = dev_data[:1000], dev_labels[:1000]
print mini_train_data.shape
print mini_train_labels.shape

(50000, 2298)
(50000,)


In [54]:
print mini_dev_data.shape
print mini_dev_labels.shape

(1000, 2298)
(1000,)


In [70]:
### K-NearestNeighbour Classifier 
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(mini_train_data,mini_train_labels)
y_pred = knn.predict(mini_dev_data)

In [71]:
print classification_report(mini_dev_labels, y_pred )

             precision    recall  f1-score   support

        0.0       0.00      0.00      0.00         0
        1.0       0.08      0.18      0.11        89
        2.0       0.00      0.00      0.00         1
        4.0       0.10      0.26      0.15        43
        5.0       0.00      0.00      0.00         7
        6.0       0.00      0.00      0.00         5
        7.0       0.22      0.33      0.27        70
        8.0       0.00      0.00      0.00         1
        9.0       0.00      0.00      0.00         1
       11.0       0.00      0.00      0.00         0
       12.0       0.07      0.05      0.06        19
       13.0       0.00      0.00      0.00        13
       15.0       0.00      0.00      0.00         1
       16.0       0.26      0.43      0.33       178
       17.0       0.00      0.00      0.00         1
       18.0       0.00      0.00      0.00         2
       19.0       0.14      0.16      0.15        31
       20.0       0.15      0.07      0.10   

In [72]:
### Multinomial Classifier
mn = MultinomialNB(alpha=0.5)
mn.fit(mini_train_data, mini_train_labels)
y_pred = mn.predict(mini_dev_data)

In [73]:
print classification_report(mini_dev_labels, y_pred )

             precision    recall  f1-score   support

        1.0       0.20      0.16      0.17        89
        2.0       0.00      0.00      0.00         1
        4.0       0.06      0.02      0.03        43
        5.0       0.00      0.00      0.00         7
        6.0       0.00      0.00      0.00         5
        7.0       0.34      0.47      0.39        70
        8.0       0.00      0.00      0.00         1
        9.0       0.00      0.00      0.00         1
       12.0       0.00      0.00      0.00        19
       13.0       0.00      0.00      0.00        13
       15.0       0.00      0.00      0.00         1
       16.0       0.28      0.65      0.39       178
       17.0       0.00      0.00      0.00         1
       18.0       0.00      0.00      0.00         2
       19.0       0.25      0.26      0.25        31
       20.0       0.17      0.11      0.13       109
       21.0       0.31      0.36      0.34       138
       23.0       0.29      0.60      0.39   

In [76]:
### Logistic Regression
lr = LogisticRegression(C=0.5)
lr.fit(mini_train_data,mini_train_labels)
y_pred = lr.predict(mini_dev_data)

In [75]:
print classification_report(mini_dev_labels, y_pred )

             precision    recall  f1-score   support

        1.0       0.16      0.13      0.15        89
        2.0       0.00      0.00      0.00         1
        4.0       0.09      0.02      0.04        43
        5.0       0.00      0.00      0.00         7
        6.0       0.00      0.00      0.00         5
        7.0       0.35      0.30      0.32        70
        8.0       0.00      0.00      0.00         1
        9.0       0.00      0.00      0.00         1
       12.0       0.00      0.00      0.00        19
       13.0       0.00      0.00      0.00        13
       15.0       0.00      0.00      0.00         1
       16.0       0.26      0.72      0.39       178
       17.0       0.00      0.00      0.00         1
       18.0       0.00      0.00      0.00         2
       19.0       0.40      0.19      0.26        31
       20.0       0.16      0.06      0.09       109
       21.0       0.28      0.46      0.35       138
       23.0       0.38      0.50      0.43   

In [77]:
print X_test.shape

(884262, 2298)


In [78]:
### Prediction for test data
y_pred = mn.predict(X_test)

In [79]:
### This function extracts the id from the test data file
def testid(data):
    
    testid = []

    for i in range(1,len(data)):
        temp_data = []
        temp_data = data[i]
        testid.append(temp_data[0])

    return testid

testid = testid(test)
print len(testid)
print testid[0]


884262
0


In [110]:
submitfile = np.zeros((len(y_pred), len(classifications)))
for i in range (len(y_pred)):
    for j in range (len(classifications)):
        if classifications[int(y_pred[i])]==classifications[j]:
            submitfile[i,j]=1
        else:
            submitfile[i,j]=0
        
print submitfile.shape
print submitfile[1]

(884262, 39)
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.]


In [112]:
test_lab_f = open("submission.csv", "w") # writing final results on a csv file

test_lab_f.write(sampleSubmission[0][0])
for j in range(len(classifications)):
    test_lab_f.write( "," + str(sampleSubmission[0][j+1]))

idx = 0
                 
for i in range(len(y_pred)):
    
    test_lab_f.write("\n")
    test_lab_f.write(str(idx))
    for j in range(len(classifications)):
        test_lab_f.write( "," + str(int(submitfile[i,j])))
    idx += 1
   

test_lab_f.close()